# Assignment 1: Analysis of the Netflix Dataset

***Dataset:*** "Netflix Movies and TV Shows" on Kaggle (`netflix_titles.csv`)  
https://www.kaggle.com/datasets/shivamb/netflix-shows is the dataset page.

This notebook analyzes the Netflix Movies and TV Shows dataset to look into trends in the streaming service's content library. Specifically, it examines how the number of titles added to Netflix has evolved over time and compares movies and TV series based on their length and content ratings.
## Data Dictionary
The Netflix titles dataset contains the following columns:
- `show_id`: Unique identifier for each title
- `type`: Indicates whether the title is a Movie or TV Show
- `title`: Name of the movie or TV show
- `director`: Director(s) of the title
- `cast`: Main cast members
- `country`: Country or countries of production
- `date_added`: Date the title was added to Netflix
- `release_year`: Year the title was originally released
- `rating`: Content rating (e.g., PG, PG-13, TV-MA)
- `duration`: Runtime in minutes for Movies or number of seasons for TV Shows
- `listed_in`: Genre(s) or category labels
- `description`: Brief description of the title
### Columns Used in This Analysis
This analysis focuses on the following columns:
- `type`
- `date_added`
- `release_year`
- `rating`
- `duration`
## Queries: 
1. How many new titles have been added to Netflix over time, broken down by year?
2. Is the average runtime or distribution of ratings different for TV shows and movies?

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("../data/netflix/netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


----
## 2. Data Cleaning 

In [2]:
def clean_duration(row):
    """
    Extracts the numeric value from the duration column.
    For Movies, returns the number of minutes (e.g., '90 min' -> 90).
    For TV Shows, returns the number of seasons (e.g., '2 Seasons' -> 2).
    Returns NaN if the duration is missing or cannot be parsed.
    """
    duration = row["duration"]
    if pd.isna(duration):
        return np.nan
    # Split the string and take the first part (the number)
    parts = str(duration).split(" ")
    try:
        return int(parts[0])
    except ValueError:
        return np.nan

In [3]:
# Drop duplicate rows if any
print("Duplicates found:", df.duplicated().sum())
df = df.drop_duplicates()

# Drop rows missing critical columns for our analysis
df = df.dropna(subset=["title", "type", "duration"])

# Convert date_added to datetime and extract year
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")
df["year_added"] = df["date_added"].dt.year

# Apply the clean_duration function to extract numeric duration
df["duration_num"] = df.apply(clean_duration, axis=1)

# Fill missing ratings with "Unknown" so they are not lost
df["rating"] = df["rating"].fillna("Unknown")

# Display cleaned dataset info
print("\nCleaned dataset shape:", df.shape)
print("\nMissing values after cleaning:")
print(df[["type", "date_added", "year_added", "duration", "duration_num", "rating"]].isna().sum())

Duplicates found: 0

Cleaned dataset shape: (8804, 14)

Missing values after cleaning:
type             0
date_added      98
year_added      98
duration         0
duration_num     0
rating           0
dtype: int64


**Note:** Missing `rating` values were filled with "Unknown" as placeholders. These are treated as a separate category and may be excluded from specific analyses when appropriate (Query 2). Rows missing `duration` were dropped since duration is essential for our second research question.

---

## 3. Analysis

### Analysis 1 — Trend Over Time: Titles Added per Year

This analysis groups titles by the year they were added to Netflix (using `date_added`) and counts how many were added each year. This helps answer **Query 1**: How has the number of titles added to Netflix changed over time?

In [4]:
# Group by year_added and count titles (ignoring rows with missing dates)
added_per_year = df.dropna(subset=["year_added"]).groupby("year_added").size()
print("Titles added per year:")
print(added_per_year)

Titles added per year:
year_added
2008.0       2
2009.0       2
2010.0       1
2011.0      13
2012.0       3
2013.0      10
2014.0      23
2015.0      73
2016.0     416
2017.0    1163
2018.0    1625
2019.0    1999
2020.0    1878
2021.0    1498
dtype: int64


### Analysis 2 — Comparison of Two Groups: Movies vs. TV Shows

This analysis compares Movies and TV Shows across two dimensions to answer **Query 2**:
1. **Duration**: Average runtime in minutes for Movies vs. average number of seasons for TV Shows.
2. **Ratings distribution**: How content ratings differ between the two types.

In [5]:
# Separate Movies and TV Shows
movies = df[df["type"] == "Movie"]
tv_shows = df[df["type"] == "TV Show"]

# Summary stats for duration by type
print("Movie duration (minutes) summary:")
print(movies["duration_num"].describe())
print("\nTV Show duration (seasons) summary:")
print(tv_shows["duration_num"].describe())

Movie duration (minutes) summary:
count    6128.000000
mean       99.577187
std        28.290593
min         3.000000
25%        87.000000
50%        98.000000
75%       114.000000
max       312.000000
Name: duration_num, dtype: float64

TV Show duration (seasons) summary:
count    2676.000000
mean        1.764948
std         1.582752
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        17.000000
Name: duration_num, dtype: float64


In [6]:
# Ratings distribution for Movies vs TV Shows (excluding "Unknown")
df_known_rating = df[df["rating"] != "Unknown"]

# Get the top ratings to keep the comparison readable
top_ratings = df_known_rating["rating"].value_counts().head(8).index.tolist()
df_top_ratings = df_known_rating[df_known_rating["rating"].isin(top_ratings)]

# Crosstab of type vs rating
rating_crosstab = pd.crosstab(df_top_ratings["rating"], df_top_ratings["type"])
print("Ratings distribution (Movies vs TV Shows):")
print(rating_crosstab)

Ratings distribution (Movies vs TV Shows):
type    Movie  TV Show
rating                
PG        287        0
PG-13     490        0
R         797        2
TV-14    1427      733
TV-MA    2062     1145
TV-PG     540      323
TV-Y      131      176
TV-Y7     139      195


---